In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [2]:
### Day and Detector settings ###
day = 160913
day2 = 101225
day3 = 111209
day4 = 121027
day5 = 130925
day6 = 141010
day7 = 150926
detector = 'n5'
data_type = 'ctime'
echan = 2



In [ ]:
### Altitude ###

pos_data = rf.poshist(day)
sat_time = pos_data[0]
sat_pos = pos_data[1]
sat_lat = pos_data[2]
sat_lon = pos_data[3]
sat_q = pos_data[4]

sat_dist = LA.norm(sat_pos, axis=0) #from the Earth's center

ell_a = 6378137
ell_b = 6356752.3142
sat_lat_rad = sat_lat*2*math.pi/360.
earth_radius_sat = ell_a * ell_b / ((ell_a**2 * np.sin(sat_lat_rad)**2 + ell_b**2 * np.cos(sat_lat_rad)**2)**0.5)

altitude = (sat_dist - earth_radius_sat)/1000

print np.mean(altitude)

plot_time_sat_date = calc.met_to_date(sat_time)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24

plt.plot(plot_time_sat, altitude, 'r-', label = 'Altitude', linewidth =2)

plt.grid()
#plt.legend()

plt.xlim([-0.5, 24.5])
plt.ylim([500, 560])

plt.title('Altitude of the Fermi Satellite on September 13th 2016', fontsize=35)
plt.xlabel('Daytime in 24h', fontsize=30)
plt.ylabel('Altitude in km', fontsize=30)

plt.tick_params(axis='both', which='major', labelsize=20)
#plt.tick_params(axis='both', which='minor', labelsize=8)

plt.show()

In [ ]:
### Longitude + Latitude ###

pos_data = rf.poshist(day)
sat_time = pos_data[0]
sat_pos = pos_data[1]
sat_lat = pos_data[2]
sat_lon = pos_data[3]
sat_q = pos_data[4]

times = []
for i in range(1, len(sat_time)):
    if np.logical_and(sat_lon[i-1] > 350, sat_lon[i] < 10):
        times.append(sat_time[i])
    #elif np.logical_and(sat_pos[0][i-1] < 0, sat_pos[0][i] > 0):
    #    print '2 ', sat_time[i]
#print sat_time[np.where(np.fabs(sat_pos[0]) < 100)]

times = np.array(times)
print (times[-1] - times[0])/((len(times)-1)*60) #Earth-orbit
print np.max(sat_lat) #inclination

In [ ]:
### J2000-Orbit ###

pos_data = rf.poshist(day)
sat_time = pos_data[0]
sat_pos = pos_data[1]
sat_lat = pos_data[2]
sat_lon = pos_data[3]
sat_q = pos_data[4]

times = []
lats = []
for i in range(1, len(sat_time)):
    if np.logical_and(sat_pos[0][i-1] > 0, sat_pos[0][i] < 0):
        times.append(sat_time[i])
        lats.append(sat_lat[i])
    #elif np.logical_and(sat_pos[0][i-1] < 0, sat_pos[0][i] > 0):
    #    print '2 ', sat_time[i]
#print sat_time[np.where(np.fabs(sat_pos[0]) < 100)]

times = np.array(times)
lats = np.array(lats)
#for i in range(1, len(times)):
#    print (times[i] - times[i-1])/60
print (times[-1] - times[0])/((len(times)-1)*60) #J2000-orbit
print lats

In [ ]:
### precession ###

month1 = np.arange(31) + 160701
month2 = np.arange(31) + 160801
month3 = np.arange(19) + 160901
days = np.append(month1, np.append(month2, month3))

times = []
lats = []

#days = np.delete(days, [13])
#print days

for day in days:
    #print day
    pos_data = rf.poshist(day)
    sat_time = pos_data[0]
    sat_pos = pos_data[1]
    sat_lat = pos_data[2]
    sat_lon = pos_data[3]
    sat_q = pos_data[4]
    
    for i in range(1, len(sat_time)):
        if np.logical_and(sat_pos[0][i-1] > 0, sat_pos[0][i] < 0):
            times.append(sat_time[i])
            lats.append(sat_lat[i])

lats = np.array(lats)
times = np.array(times)

precession = times[np.where(np.logical_and(lats >= -0.1, lats <= 0.1))]

precession_time = (precession[2] - precession[0])/3600/24
print precession_time

In [ ]:
print np.where(np.logical_and(lats >= -0.3, lats <= 0.3))
print lats[np.where(np.logical_and(lats >= -0.3, lats <= 0.3))]
print times[np.where(np.logical_and(lats >= -0.1, lats <= 0.1))]
precession = times[np.where(np.logical_and(lats >= -0.1, lats <= 0.1))]

#print precession
#precession = np.array(precession)
precession_time = (precession[2] - precession[0])/3600/24
print precession_time

In [ ]:
### Fermi-Satellite Orbital-behaviour ###

days = np.array([101225, 111209, 121027, 130925, 141010, 150926, 160913])
years = np.array([2010, 2011, 2012, 2013, 2014, 2015, 2016])
str_years = np.array(['2010', '2011', '2012', '2013', '2014', '2015', '2016'])
altitude = np.array([550.645, 548.651, 546.451, 544.102, 539.703, 535.571, 533.763])
j2000_orb = np.array([95.575, 95.532, 95.489, 95.458, 95.346, 95.268, 95.250])
earth_orb = np.array([102.44, 102.39, 102.30, 102.28, 102.10, 102.03, 102.05])
inclination = np.array([25.55, 25.57, 25.55, 25.50, 25.55, 25.53, 25.66])

plt.plot(years, altitude, 'r-', label = 'Altitude in km', linewidth =2)
#plt.plot(years, inclination, 'y-', label = 'Inclination in degrees')
#plt.plot(years, j2000_orb, 'b-', label = 'J2000 orbital period in min')
#plt.plot(years, earth_orb, 'g-', label = 'Geographic orbital period in min')

plt.grid()
#plt.legend()

plt.xlim([2010, 2016])
plt.ylim([519, 560])

plt.xticks(years, str_years)
plt.title('Altitude of the Fermi Satellite', fontsize=35)
plt.xlabel('Year', fontsize=30)
plt.ylabel('Altitude in km', fontsize=30)

plt.tick_params(axis='both', which='major', labelsize=20)
#plt.tick_params(axis='both', which='minor', labelsize=8)

plt.show()

In [ ]:
### Time conversion tests ###

met = 448588740.740073
day = 150321

data = calc.met_to_date(met)
mjdutc = data[0]
mjdtt = data[1]
isot = data[2]
date = data[3]
decimal = data[4]

print mjdutc
print mjdtt
print isot
print date
print decimal

sun = calc.sun_pos(day)

In [ ]:
### Angular calibration for ctime channels ###

fitsname = 'peak_eff_area_angle_calib_GBM_all.fits'
user = getpass.getuser()
fits_path = '/home/' + user + '/Work/calibration/'
fitsfilepath = os.path.join(fits_path, fitsname)
fitsfile = fits.open(fitsfilepath, mode='update')
data = fitsfile[1].data
fitsfile.close()
x = data.field(0)
y1 = data.field(1)#for NaI (33 keV)
y2 = data.field(2)#for NaI (279 keV)
y3 = data.field(3)#for NaI (662 keV)
y4 = data.field(4)#for BGO (898 keV)
y5 = data.field(5)#for BGO (1836 keV)
        
ang_eff = []

echans_energy = np.array([8, 19, 38, 75, 198, 417, 762, 1492])

#ctime linear-interpolation factors
#y1_fac = np.array([1.2, 1.1, 1., 1., 1., 0., 0., 0.])
#y2_fac = np.array([0., 0., 5./279., 42./279., 165./279., 1., 0., 0.])
#y3_fac = np.array([0., 0., 0., 0., 0., 138./662., 1., 1.])

### Angepasst ###
#y1_fac = np.array([1.2, 1.08, 238./246., 196./246., 77./246., 0., 0., 0.])
#y2_fac = np.array([0., 0., 5./246., 40./246., 159./246., 230./383., 0., 0.])
#y3_fac = np.array([0., 0., 0., 0., 0., 133./383., .7, .5])

###Angepasst 2 ###
y1_fac = np.array([1.2, 1.08, 238./246., 196./246., 127./246., 0., 0., 0.])
y2_fac = np.array([0., 0., 5./246., 40./246., 109./246., 230./383., 0., 0.])
y3_fac = np.array([0., 0., 0., 0., 0., 133./383., .7, .5])

mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k', '0.5']

for i in range(0, len(echans_energy)):
    #resulting effective area curve
    y = y1_fac[i]*y1 + y2_fac[i]*y2 + y3_fac[i]*y3
    
    #normalize
    y = y
    
    plt.plot(x, y, label = 'Energy channel ' + str(i))

plt.plot(x, y1, 'k--', label = '33 keV')
plt.plot(x, y2, 'b--', label = '279 keV')
plt.plot(x, y3, 'g--', label = '662 keV')
plt.xlabel('Angle in degrees', fontsize=20)
plt.ylabel('Normalized photopeak effective area', fontsize=20)
plt.grid()
plt.legend()
plt.show()

In [ ]:
y_new = np.array([y1[90], y2[90], y3[90]])
x_new = np.array([33, 279, 662])

### Schnittpunkte ###
#y1_fac = np.array([1.2, 1.1, 241./246., 204./246., 81./246., 0., 0., 0.])
#y2_fac = np.array([0., 0., 5./246., 42./246., 165./246., 245./383., 0., 0.])
#y3_fac = np.array([0., 0., 0., 0., 0., 138./383., .7, .5])

### Angepasst ###
#y1_fac = np.array([1.2, 1.08, 238./246., 196./246., 77./246., 0., 0., 0.])
#y2_fac = np.array([0., 0., 5./246., 40./246., 159./246., 230./383., 0., 0.])
#y3_fac = np.array([0., 0., 0., 0., 0., 133./383., .7, .5])

###Angepasst 2 ###
y1_fac = np.array([1.2, 1.08, 238./246., 196./246., 127./246., 0., 0., 0.])
y2_fac = np.array([0., 0., 5./246., 40./246., 109./246., 230./383., 0., 0.])
y3_fac = np.array([0., 0., 0., 0., 0., 133./383., .7, .5])

y_echans = y1_fac*y1[90] + y2_fac*y2[90] + y3_fac*y3[90]

plt.plot(x_new, y_new/y1[90], 'k--')
plt.plot(echans_energy, y_echans/y1[90], 'g-')
plt.xlabel('Energy in keV', fontsize=20)
plt.ylabel('Normalized photopeak effective area maximum', fontsize=20)
plt.xlim([0, 1600])
plt.show()

In [ ]:
### Earth occultation vs. Earth angle comparison ###

#day = 150926
#detector = 'n5'
#echan = 2
#data_type = 'ctime'

year = int('20' + str(day)[0:2])

#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))

#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

#read the measurement data
ctime_data = rf.ctime(detector, day)
echans = ctime_data[0]
total_counts = ctime_data[1]
echan_counts = ctime_data[2]
total_rate = ctime_data[3]
echan_rate = ctime_data[4]
bin_time = ctime_data[5]
good_time = ctime_data[6]
exptime = ctime_data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

counts = echan_rate[echan]

#read the satellite data
sat_data = rf.poshist_bin(day, bin_time_mid, detector, data_type)
sat_time_bin = sat_data[0]
sat_pos_bin = sat_data[1]
sat_lat_bin = sat_data[2]
sat_lon_bin = sat_data[3]
sat_q_bin = sat_data[4]

#calculate the earth data
earth_data = calc.earth_ang_bin(detector, day, bin_time_mid, data_type)
earth_ang_bin = earth_data[0]
earth_ang_eff = calc.ang_eff(earth_ang_bin, echan)[0]
earth_occ_bin = calc.earth_occ_eff(earth_ang_bin, echan)

earth_ang_bin[np.where(total_rate == 0)] = 0
earth_ang_eff[np.where(total_rate == 0)] = 0
earth_occ_bin[np.where(total_rate == 0)] = 0


#### plotting ####
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours
plot_time_sat_date = calc.met_to_date(sat_time_bin)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours


###plot each on the same axis as converted to counts###
fig, ax1 = plt.subplots()

#add two independent y-axes
ax2 = ax1.twinx()
ax3 = ax1.twinx()
axes = [ax1, ax2, ax3]

#Make some space on the right side for the extra y-axis
fig.subplots_adjust(right=0.75)

# Move the last y-axis spine over to the right by 20% of the width of the axes
#axes[-1].spines['right'].set_position(('axes', 1.2))

# To make the border of the right-most axis visible, we need to turn the frame on. This hides the other plots, however, so we need to turn its fill off.
#axes[-1].set_frame_on(True)
#axes[-1].patch.set_visible(False)

plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Counts', alpha=0.2)
plot3 = ax3.plot(plot_time_sat, 20 + 1.8*earth_ang_bin, 'r-', label = 'Earth-detector angle', linewidth=2)
plot4 = ax3.plot(plot_time_sat, 240 - 5*earth_ang_eff, 'g-', label = 'effective Earth-detector angle', linewidth=2)
plot2 = ax2.plot(plot_time_sat, 1.3 - earth_occ_bin, 'c-', label = 'Earth occultation', linewidth=2)

plots = plot1 + plot2 + plot3 + plot4
labels = [l.get_label() for l in plots]
ax3.legend(plots, labels, loc=1, fontsize=15)

ax1.grid()

ax1.set_xlabel('Time of day in 24h', fontsize=20)
ax1.set_ylabel('Number of counts', fontsize=20)
#ax2.set_ylabel('Effective area fraction', fontsize=20)
#ax3.set_ylabel('Angle in degrees', fontsize=20)

ax2.axes.get_yaxis().set_visible(False)
ax3.axes.get_yaxis().set_visible(False)

ax1.set_xlim([2.4, 10.3])
ax1.set_ylim([70, 360])
#ax2.set_xlim([0, 24.1])
ax2.set_ylim([0.3, 1.6])
#ax3.set_xlim([0, 24.1])
ax3.set_ylim([20, 320])
#ax4.set_xlim([0, 24.1])
#ax4.set_ylim([-1.5, 1.5])

ax1.tick_params(axis='both', which='major', labelsize=15)
ax2.tick_params(axis='both', which='major', labelsize=15)
ax3.tick_params(axis='both', which='major', labelsize=15)

plt.title('Countrate, Earth occultation and angles for the ' + ordinal(int(str(echan))) + ' energy-channel of the ' + detector + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=30)

plt.show()

In [ ]:
### CTIME-data & SAA plot ###

data = rf.ctime(detector, day)
chan_energies = data[0]
total_counts = data[1]
echan_counts = data[2]
total_rate = data[3]
echan_rate = data[4]
bin_time = data[5]
good_time = data[6]
exptime = data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

counts = echan_rate[echan]

year = int('20' + str(day)[0:2])
#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))
#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])


#### plotting ####
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours


plt.plot(plot_time_bin, counts, 'b-', label = 'Countrate')

plt.legend(fontsize=15)
plt.grid()

plt.xlim([21.5, 24])

plt.xlabel('Time of day in 24h', fontsize=20)
plt.ylabel('Countrate', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=15)

plt.title(data_type + '-countrate for the ' + ordinal(int(str(echan))) + ' energy-channel of the ' + detector + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=30)

plt.show()

In [ ]:
### McIlwain L-parameter plot ###

echan = 6

#read the measurement data
ctime_data = rf.ctime(detector, day)
chan_energies = ctime_data[0]
total_counts = ctime_data[1]
echan_counts = ctime_data[2]
total_rate = ctime_data[3]
echan_rate = ctime_data[4]
bin_time = ctime_data[5]
good_time = ctime_data[6]
exptime = ctime_data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

counts = echan_rate[echan]

year = int('20' + str(day)[0:2])
#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))
#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

#read the satellite data
sat_data = rf.poshist_bin(day, bin_time_mid, detector, data_type)
sat_time_bin = sat_data[0]
sat_pos_bin = sat_data[1]
sat_lat_bin = sat_data[2]
sat_lon_bin = sat_data[3]
sat_q_bin = sat_data[4]

#read the mcilwain parameter data
sat_time = readfile.poshist(readfile(), day)[0]
lat_data = readfile.mcilwain(readfile(), day)
mc_b = lat_data[1]
mc_l = lat_data[2]
mc_b = calculate.intpol(calculate(), mc_b, day, 0, sat_time, bin_time_mid)[0]
mc_l = calculate.intpol(calculate(), mc_l, day, 0, sat_time, bin_time_mid)[0]

magnetic = mc_l
magnetic = magnetic - np.mean(magnetic, dtype=np.float64)
magnetic[np.where(total_rate == 0)] = 0


#### plotting ####
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours
plot_time_sat_date = calc.met_to_date(sat_time_bin)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours


###plot each on the same axis as converted to counts###
fig, ax1 = plt.subplots()

#add two independent y-axes
ax2 = ax1.twinx()
axes = [ax1, ax2]

#Make some space on the right side for the extra y-axis
#fig.subplots_adjust(right=0.75)

# Move the last y-axis spine over to the right by 20% of the width of the axes
#axes[-1].spines['right'].set_position(('axes', 1.2))

# To make the border of the right-most axis visible, we need to turn the frame on. This hides the other plots, however, so we need to turn its fill off.
#axes[-1].set_frame_on(True)
#axes[-1].patch.set_visible(False)

plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Counts', alpha=0.2)
plot2 = ax2.plot(plot_time_sat, 0.9*magnetic, 'g-', label = 'McIlwain L-parameter', linewidth=2)

plots = plot1 + plot2
labels = [l.get_label() for l in plots]
ax2.legend(plots, labels, loc=1, fontsize=15)

ax1.grid()

ax1.set_xlabel('Time of day in 24h', fontsize=20)
ax1.set_ylabel('Number of counts', fontsize=20)
#ax2.set_ylabel('Effective area fraction', fontsize=20)

ax2.axes.get_yaxis().set_visible(False)

ax1.set_xlim([15, 20])
#ax1.set_xlim([-0.5, 24.5])
ax1.set_ylim([0, 230])
#ax2.set_xlim([0, 24.1])
ax2.set_ylim([-0.5, 1])

ax1.tick_params(axis='both', which='major', labelsize=15)
ax2.tick_params(axis='both', which='major', labelsize=15)

plt.title('Countrate and the McIlwain L-parameter for the ' + ordinal(int(str(echan))) + ' energy-channel of the ' + detector + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=30)

plt.show()

print len(counts)

In [ ]:
### CGB from Ajello paper ###

energies1 = np.arange(10000) + 1

C1 = 0.1015

E_B1 = 29.99

R_11 = 1.32

R_21 = 2.88

spectrum1 = (energies1**2)*C1/((energies1/E_B1)**R_11 + (energies1/E_B1)**R_21)
intensity1 = C1/((energies1/E_B1)**R_11 + (energies1/E_B1)**R_21)

#plt.plot(energies1, spectrum1, label = 'CGB', linewidth = 2)
plt.plot(energies1, intensity1, label = 'CGB', linewidth = 2)



#energies2 = np.arange(10000) + 1

#C2 = 0.109

#E_B2 = 28.

#R_12 = 1.40

#R_22 = 2.88

#spectrum2 = (energies2**2)*C2/((energies2/E_B2)**R_12 + (energies2/E_B2)**R_22)

#plt.plot(energies2, spectrum2, label = 'CGB Tuerler 2010', linewidth = 2)



energies3 = np.arange(10000) + 1

C3 = 0.0148

E_B3 = 33.7

R_13 = -5.

R_23 = 1.72

spectrum3 = (energies3**2)*C3/((energies3/E_B3)**R_13 + (energies3/E_B3)**R_23)
intensity3 = C3/((energies3/E_B3)**R_13 + (energies3/E_B3)**R_23)

#plt.plot(energies3, spectrum3, label = 'Earth albedo', linewidth = 2)
plt.plot(energies3, intensity3, label = 'Earth albedo', linewidth = 2)



earth_effect = np.fabs(intensity3 - intensity1)

zero = energies3[np.where(earth_effect < 0.0001)]
print zero[:15]

#plt.plot(energies1, earth_effect, label = 'Earth occultation effect', linewidth = 2)


plt.xscale('log')
plt.yscale('log')
plt.legend(fontsize=15)
plt.grid()

plt.xlim(10, 1000)
plt.ylim(0.0001, 1)

plt.xlabel('Energy in keV', fontsize=20)
plt.ylabel('Intensity in [cm$^{-2}$ s$^{-1}$ keV$^{-1}$ sr$^{-1}$]', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=15)

plt.title('CGB intensity spectrum vs. the Earth albedo intesity spectrum', fontsize=30)

plt.show()

In [ ]:
### CTIME-Channels gewichteter Mittelwert ###

#Read the data file
user = getpass.getuser()
path = '/home/' + user + '/Documents/Plot_digitalizer/'
filepath = os.path.join(path, 'NaI_total.dat')
f1 = open(filepath)
#store the file contents in a list of every row
lines = f1.readlines()[1:]
f1.close()

#just some variables going to be arrays of our data
x1 = []
y1 = []

#scan each line in the list, split the string into two and add them to the arrays
for line in lines:
    p = line.split()
    x1.append(float(p[0]))
    y1.append(float(p[1]))

xv = np.array(x1)
yv = np.array(y1)

energies = np.arange(1997) + 4

tck = interpolate.splrep(xv, yv, s=1.12)
area = interpolate.splev(energies, tck, der=0)

area[:4] = np.array([0.058, 0.237, 0.802, 2.212])

#plt.plot(xv, yv, 'r--')
#plt.plot(energies, area, 'b-')

#labeling the axes
#plt.ylabel('Effective Area in cm^2')
#plt.xlabel('Energy in keV')

#plt.ylim(0.01, 30)

#plt.xscale('log')
#plt.yscale('log')

#anzeigen lassen
#plt.show()

energy_boundaries = np.array([985,2000])

chan_en = energies[np.where(np.logical_and(energies >= energy_boundaries[0], energies <= energy_boundaries[1]))]

chan_ar = area[np.where(np.logical_and(energies >= energy_boundaries[0], energies <= energy_boundaries[1]))]

chan_mid = np.sum((chan_ar)*(chan_en))/((np.sum(chan_ar)))

print chan_mid






In [ ]:
### CTIME angular dependency from DRM ###

fitsname = 'peak_eff_area_angle_calib_GBM_all_DRM.fits'
user = getpass.getuser()
fits_path = '/home/' + user + '/Work/calibration/'
fitsfilepath = os.path.join(fits_path, fitsname)
fitsfile = fits.open(fitsfilepath, mode='update')
data = fitsfile[1].data
fitsfile.close()
x0 = data.field(0)
y0 = data.field(1)#for NaI (4-12 keV) gewichteter Mittelwert = 10.76 keV
y1 = data.field(2)#for NaI (12-27 keV) gewichteter Mittelwert = 20.42 keV
y2 = data.field(3)#for NaI (27-50 keV) gewichteter Mittelwert = 38.80 keV
y3 = data.field(4)#for NaI (50-102 keV) gewichteter Mittelwert = 76.37 keV
y4 = data.field(5)#for NaI (102-295 keV) gewichteter Mittelwert = 190.19 keV
y5 = data.field(6)#for NaI (295-540 keV) gewichteter Mittelwert = 410.91 keV
y6 = data.field(7)#for NaI (540-985 keV) gewichteter Mittelwert = 751.94 keV
y7 = data.field(8)#for NaI (985-2000 keV) gewichteter Mittelwert = 1466.43 keV

x = ((np.arange(3601) - 1800.)/10.)

tck = interpolate.splrep(x0, y0)
y0 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y1)
y1 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y2)
y2 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y3)
y3 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y4)
y4 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y5)
y5 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y6)
y6 = interpolate.splev(x, tck, der=0)
tck = interpolate.splrep(x0, y7)
y7 = interpolate.splev(x, tck, der=0)



mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k', '0.5']

plt.plot(x, y0, label = '0')
plt.plot(x, y1, label = '1')
plt.plot(x, y2, label = '2')
plt.plot(x, y3, label = '3')
plt.plot(x, y4, label = '4')
plt.plot(x, y5, label = '5')
plt.plot(x, y6, label = '6')
plt.plot(x, y7, label = '7')



plt.tick_params(axis='both', which='major', labelsize=15)

plt.xlabel('Angle in degrees', fontsize=20)
plt.ylabel('Effective area in cm$^2$', fontsize=20)
plt.grid()
plt.legend(fontsize=15)
plt.show()

In [5]:
### DDBF Plot ###

day = 150926

sources_data = rf.point_sources()
sources_names = sources_data[0]
sources_coordinates = sources_data[1]
sources_number = int(len(sources_names))

year = int('20' + str(day)[0:2])

#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))

#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

detector_name = 'n5'
echan = 2
data_type = 'ctime'

#read the measurement data
ctime_data = rf.ctime(detector_name, day)
chan_energies = ctime_data[0]
total_counts = ctime_data[1]
echan_counts = ctime_data[2]
total_rate = ctime_data[3]
echan_rate = ctime_data[4]
bin_time = ctime_data[5]
good_time = ctime_data[6]
exptime = ctime_data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

total_rate = np.sum(echan_rate[1:-1], axis = 0)

counts = echan_rate[echan]

#read the satellite data
sat_data = rf.poshist_bin(day, bin_time_mid, detector_name, data_type)
sat_time_bin = sat_data[0]
sat_pos_bin = sat_data[1]
sat_lat_bin = sat_data[2]
sat_lon_bin = sat_data[3]
sat_q_bin = sat_data[4]

#calculate the sun data
sun_data = calc.sun_ang_bin(detector_name, day, bin_time_mid, data_type)
sun_ang_bin = sun_data[0]
sun_ang_bin = calc.ang_eff(sun_ang_bin, echan)[0]
sun_rad = sun_data[2]
sun_ra = sun_rad[:,0]
sun_dec = sun_rad[:,1]
sun_occ = calc.src_occultation_bin(day, sun_ra, sun_dec, bin_time_mid)[0]

#sun_max = calc.ang_eff(0., echan, data_type)[0]
sun_max = 126.6768698 

#calculate the earth data
earth_data = calc.earth_ang_bin(detector_name, day, bin_time_mid, data_type)
earth_ang_bin = earth_data[0]
#earth_ang_bin = calc.ang_eff(earth_ang_bin, echan)[0]
earth_ang_bin = calc.earth_occ_eff(earth_ang_bin, echan)

earth_max = calc.earth_occ_eff(0., echan)

#calculate the pointsources data
sources_ang_bin = np.array([np.zeros(len(sun_ang_bin))])

for i in range(0, sources_number):
    src_ra = sources_coordinates[0,i]
    src_dec = sources_coordinates[1,i]
    
    src_data = calc.burst_ang_bin(detector_name, day, src_ra, src_dec, bin_time_mid, data_type)
    src_ang_bin = src_data[0]
    src_ang_bin = calc.ang_eff(src_ang_bin, echan, data_type)[0]
    src_occ = calc.src_occultation_bin(day, src_ra, src_dec, bin_time_mid, detector_name, data_type)[0]
    
    src_ang_bin[np.where(src_occ == 0)] = 0
    src_ang_bin[np.where(total_rate == 0)] = 0
    
    #remove vertical movement from scaling the src_ang_bin functions
    #src_ang_bin[src_ang_bin>0] = src_ang_bin[src_ang_bin>0] - np.min(src_ang_bin[src_ang_bin>0])
    
    src_ang_bin = np.array(src_ang_bin)
    
    sources_ang_bin = np.concatenate((sources_ang_bin, [src_ang_bin]), axis = 0)

sources_ang_bin = np.delete(sources_ang_bin, 0, 0)

sources_frac_bin = sources_ang_bin/sun_max

#read the SFL data
flares = rf.flares(year)
flares_day = flares[0]
flares_time = flares[1]
if np.any(flares_day == day) == True:
    flares_today = flares_time[:,np.where(flares_day == day)]
    flares_today = np.squeeze(flares_today, axis=(1,))/3600.
else:
    flares_today = np.array(-5)

#read the mcilwain parameter data
sat_time = readfile.poshist(readfile(), day)[0]
lat_data = readfile.mcilwain(readfile(), day)
mc_b = lat_data[1]
mc_l = lat_data[2]

mc_b = calc.intpol(mc_b, day, 0, sat_time, bin_time_mid)[0]
mc_l = calc.intpol(mc_l, day, 0, sat_time, bin_time_mid)[0]

magnetic = mc_l
#magnetic = magnetic - np.mean(magnetic, dtype=np.float64)

magnetic_max = 3.

#constant function corresponding to the diffuse y-ray background
cgb = np.ones(len(total_rate))

cgb_max = 2.

#counts[120000:] = 0
cgb[np.where(total_rate == 0)] = 0
earth_ang_bin[np.where(total_rate == 0)] = 0
sun_ang_bin[np.where(sun_occ == 0)] = 0
sun_ang_bin[np.where(total_rate == 0)] = 0
magnetic[np.where(total_rate == 0)] = 0

sources_curve = np.squeeze(np.sum(sources_ang_bin, axis = 0))
sources_frac_curve = np.squeeze(np.sum(sources_frac_bin, axis = 0))/sources_number


#convert the x-axis into hours of the day
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours
plot_time_sat_date = calc.met_to_date(sat_time_bin)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours


fig, ax1 = plt.subplots()

#plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Countrate')
#plot2 = ax1.plot(plot_time_sat, sun_ang_bin, 'y-', label = 'Sun effective angle')
#plot3 = ax1.plot(plot_time_sat, earth_ang_bin, 'c-', label = 'Earth occultation')
#plot4 = ax1.plot(plot_time_sat, magnetic, 'g-', label = 'McIlwain L-parameter')
#plot5 = ax1.plot(plot_time_sat, cgb, 'b--', label = 'CGB')
#plot6 = ax1.plot(plot_time_sat, sources_curve, 'k-', label = 'Point-like sources')

#plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Countrate')
plot2 = ax1.plot(plot_time_sat, sun_ang_bin/sun_max, 'y-', label = 'Sun effective angle')
plot3 = ax1.plot(plot_time_sat, earth_ang_bin/earth_max, 'c-', label = 'Earth occultation')
plot4 = ax1.plot(plot_time_sat, magnetic/magnetic_max, 'g-', label = 'McIlwain L-parameter')
plot5 = ax1.plot(plot_time_sat, cgb/cgb_max, 'b--', label = 'CGB')
plot6 = ax1.plot(plot_time_sat, sources_frac_curve, 'k-', label = 'Point-like sources')

#plot vertical lines for the solar flares of the day
if np.all(flares_today != -5):
    if len(flares_today[0]) > 1:
        for i in range(0, len(flares_today[0])):
            plt.axvline(x = flares_today[0,i], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
            plt.axvline(x = flares_today[1,i], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')
    else:
        plt.axvline(x = flares_today[0], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
        plt.axvline(x = flares_today[1], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')

plots = plot2 + plot3 + plot4 + plot5 + plot6
labels = [l.get_label() for l in plots]
ax1.legend(plots, labels, loc=1, fontsize=15)

ax1.grid()

ax1.set_xlabel('Time of day in 24h', fontsize=20)
ax1.set_ylabel('Countrate', fontsize=20)
ax1.tick_params(axis='both', which='major', labelsize=15)

#ax1.set_xlim([9.84, 9.85])
ax1.set_xlim([-0.5, 24.5])
#ax1.set_ylim([-0.5, 1.5])

plt.title('blubb', fontsize=30)

plt.show()

In [ ]:
### IceCube Neutrino Event Winkel ### eigentlich Sonnen-Event

detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5'])

day = 140611
burst_ra = 110.337
burst_dec = 11.484

start_time = 4.8
end_time = 5.0

v_line = 4.90648687098


burst_occ = calc.src_occultation(day, burst_ra, burst_dec)[0]

sat_time = rf.poshist(day)[0]

plot_time_sat_date = calc.met_to_date(sat_time)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours

mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

for i in range(0, len(detectors)):
    #print detectors[i]
    det_data = calc.burst_ang(detectors[i], day, burst_ra, burst_dec)
    det_ang = det_data[0]
    #sat_time = det_data[1]
    det_ang[np.where(det_ang > 90.)] = 0.
    
    plt.plot(plot_time_sat, det_ang, label = detectors[i])
    
plt.axvline(x = v_line, ymin = 0., ymax = 1., linewidth=2, color = 'grey')

plt.plot(plot_time_sat, burst_occ, label = 'Earth')    
plt.legend(fontsize=15)
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=15)

plt.xlabel('Daytime in 24h', fontsize=20)
plt.ylabel('Angle in degrees', fontsize=20)
plt.title('Winkelverhaeltnisse zwischen dem IceCube Neutrino-Event und den Fermi-Detektoren', fontsize=30)

plt.xlim([start_time, end_time])
plt.ylim([-10, 150])

plt.show()

In [ ]:
### GW Event Winkel ###

detectors = np.array(['n2', 'n4', 'n5', 'n8', 'na', 'nb'])
#detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb'])

day = 150914
burst_ra = 75
burst_dec = -73

start_time = 9.75
end_time = 9.95

v_line = 9.84605472222


burst_occ = calc.src_occultation(day, burst_ra, burst_dec)[0]

sat_time = rf.poshist(day)[0]

plot_time_sat_date = calc.met_to_date(sat_time)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours

mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

for i in range(0, len(detectors)):
    #print detectors[i]
    det_data = calc.burst_ang(detectors[i], day, burst_ra, burst_dec)
    det_ang = det_data[0]
    #sat_time = det_data[1]
    #det_ang[np.where(det_ang > 90.)] = 0.
    
    plt.plot(plot_time_sat, det_ang, label = detectors[i])
    
plt.axvline(x = v_line, ymin = 0., ymax = 1., linewidth=2, color = 'grey')

plt.plot(plot_time_sat, burst_occ, label = 'Earth')    
plt.legend(fontsize=15)
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=15)

plt.xlabel('Daytime in 24h', fontsize=20)
plt.ylabel('Angle in degrees', fontsize=20)
plt.title('Winkelverhaeltnisse zwischen dem IceCube Neutrino-Event und den Fermi-Detektoren', fontsize=30)

plt.xlim([start_time, end_time])
plt.ylim([-10, 180])

plt.show()